# PyTorch - 深度学习框架

PyTorch是当前最流行的深度学习框架之一，以动态计算图和易用性著称。

## 本教程完整内容
1. PyTorch基础 - 张量操作
2. 自定义神经网络模型
3. 自定义数据集类
4. 训练、验证、测试完整流程
5. 模型保存和恢复（断点续训）
6. TensorBoard可视化
7. 其他日志记录工具

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from datetime import datetime
import logging

print(f"PyTorch版本: {torch.__version__}")
print(f"Torchvision版本: {torchvision.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA设备数量: {torch.cuda.device_count()}")
    print(f"当前CUDA设备: {torch.cuda.get_device_name(0)}")

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")